In [1]:
from meerkat import ds3231, parser

import pandas as pd

from importlib import reload

In [2]:
rtc = ds3231.DS3231(bus_n=1)

In [3]:
rtc.get_time()

(2020, 10, 28, 10, 53, 28)

In [4]:
rtc.get_temp()

23.75

In [5]:
rtc.publish(description="test_1", n=1)

'{"description": "test_1", "sample_n": 0, "rtc_time": "2020-10-28 10:53:28", "temp_C": 23.75, "std_time_ms": "2020-10-28 10:54:31.605877"}'

In [6]:
rtc.publish(description="test_1", n=2, delay=3)

['{"description": "test_1", "sample_n": 0, "rtc_time": "2020-10-28 10:53:28", "temp_C": 23.75, "std_time_ms": "2020-10-28 10:54:31.623933"}',
 '{"description": "test_1", "sample_n": 0, "rtc_time": "2020-10-28 10:53:31", "temp_C": 23.75, "std_time_ms": "2020-10-28 10:54:34.629217"}']

In [7]:
# default writer format is CSV, switch to JSON
rtc.writer_output = 'json'

In [8]:
# writer method with description and sample number
rtc.write(description='test_2', n=6, delay=1)

TypeError: can only concatenate str (not "NoneType") to str

In [ ]:
rtc.json_writer.path

In [ ]:
with open(rtc.json_writer.path, 'r') as f:
    for _ in range(4):
        print(f.readline().strip())

In [ ]:
# switch back to CSV format
rtc.writer_output = 'csv'

In [ ]:
rtc.write(description='test_3', n=6, delay=1)

In [ ]:
m, df = parser.csv_resource(rtc.csv_writer.path)

In [ ]:
df["rtc_datetime64_ns"] = pd.to_datetime(df.rtc_time)
df.temp_C = df.temp_C.astype(float)

In [ ]:
df.dtypes

In [ ]:
df

In [ ]:
df[["rtc_datetime64_ns", "temp_C"]].plot(x="rtc_datetime64_ns", marker="x");